# Example: Pump and Valve Simulation

This notebook demonstrates a simulation of a system with a `Pump` and a `Valve`. It showcases how to script specific events at particular times in a simulation by manually publishing messages to the `MessageBus`.

The system pumps water from a `Source Reservoir` to a `Destination Reservoir` through a `Pipe` and `Valve`.

## Simulation Logic

Instead of using a predefined control agent, this simulation uses a custom loop to issue commands at specific times:
- **t = 5s**: The pump is turned on.
- **t = 10s**: The valve is opened to 50%.
- **t = 100s**: The valve is opened fully to 100%.
- **t = 180s**: The pump is turned off and the valve is closed.

This allows us to observe the direct impact of these components on the system's state.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from swp.core_engine.testing.simulation_harness import SimulationHarness
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.simulation_identification.physical_objects.pipe import Pipe
from swp.simulation_identification.physical_objects.valve import Valve
from swp.simulation_identification.physical_objects.pump import Pump

# 1. Setup
config = {'duration': 200, 'dt': 1.0}
harness = SimulationHarness(config)
message_bus = harness.message_bus

# 2. Components
source_reservoir = Reservoir(name="source_res", initial_state={'volume': 50000, 'water_level': 5.0}, parameters={'surface_area': 10000})
dest_reservoir = Reservoir(name="dest_res", initial_state={'volume': 25000, 'water_level': 5.0}, parameters={'surface_area': 5000})
pump = Pump(name="pump1", initial_state={'status': 0, 'outflow': 0}, parameters={'max_flow_rate': 5, 'max_head': 20, 'power_consumption_kw': 75}, message_bus=message_bus, action_topic="action.pump1.status")
pipe = Pipe(name="pipe1", initial_state={'outflow': 0}, parameters={'length': 100, 'diameter': 0.5, 'friction_factor': 0.02})
valve = Valve(name="valve1", initial_state={'opening': 0}, parameters={'diameter': 0.5, 'discharge_coefficient': 0.9}, message_bus=message_bus, action_topic="action.valve1.opening")

harness.add_component(source_reservoir)
harness.add_component(pump)
harness.add_component(pipe)
harness.add_component(valve)
harness.add_component(dest_reservoir)

# 3. Topology
harness.add_connection("source_res", "pump1")
harness.add_connection("pump1", "pipe1")
harness.add_connection("pipe1", "valve1")
harness.add_connection("valve1", "dest_res")

harness.build()

# 4. Custom Simulation Loop
num_steps = int(harness.duration / harness.dt)
history = []

import sys
original_stdout = sys.stdout
with open('simulation_log.txt', 'w') as f:
    sys.stdout = f
    for i in range(num_steps):
        current_time = i * harness.dt

        # Scripted events
        if current_time == 5:
            message_bus.publish("action.pump1.status", {'control_signal': 1})
        if current_time == 10:
            message_bus.publish("action.valve1.opening", {'control_signal': 0.5})
        if current_time == 100:
            message_bus.publish("action.valve1.opening", {'control_signal': 1.0})
        if current_time == 180:
            message_bus.publish("action.pump1.status", {'control_signal': 0})
            message_bus.publish("action.valve1.opening", {'control_signal': 0})

        # Step the model
        harness._step_physical_models(harness.dt)
        
        # Record history
        step_history = {'time': current_time}
        for cid in harness.sorted_components:
            step_history[cid] = harness.components[cid].get_state()
        history.append(step_history)

sys.stdout = original_stdout
harness.history = history # Store history in harness for consistency
print("Pump-valve simulation complete.")

## Results and Visualization

The plots show the water levels in the two reservoirs, the flow rate in the pipe, and the status of the pump and valve, clearly illustrating the consequences of the scripted control actions.

In [ ]:
# Extract data from history
time = [h['time'] for h in harness.history]
source_res_level = [h['source_res']['water_level'] for h in harness.history]
dest_res_level = [h['dest_res']['water_level'] for h in harness.history]
pump_status = [h['pump1']['status'] for h in harness.history]
valve_opening = [h['valve1']['opening'] for h in harness.history]
pipe_outflow = [h['pipe1']['outflow'] for h in harness.history]

# Create a DataFrame
df = pd.DataFrame({
    'Time': time,
    'Source Level': source_res_level,
    'Destination Level': dest_res_level,
    'Pump Status': pump_status,
    'Valve Opening': valve_opening,
    'Pipe Outflow': pipe_outflow
})

print(df.head())

# Plot the results
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 15), sharex=True)

# Reservoir Levels
ax1.plot(df['Time'], df['Source Level'], label='Source Reservoir Level')
ax1.plot(df['Time'], df['Destination Level'], label='Destination Reservoir Level')
ax1.set_ylabel('Water Level (m)')
ax1.set_title('Pump-Valve Simulation Results')
ax1.grid(True)
ax1.legend()

# Pump and Valve Status
ax2.plot(df['Time'], df['Pump Status'], label='Pump Status (1=On)', drawstyle='steps-post')
ax2.plot(df['Time'], df['Valve Opening'], label='Valve Opening (%)', drawstyle='steps-post')
ax2.set_ylabel('Status / Opening')
ax2.grid(True)
ax2.legend()

# Pipe Outflow
ax3.plot(df['Time'], df['Pipe Outflow'], label='Pipe Outflow', color='green')
ax3.set_xlabel('Time (s)')
ax3.set_ylabel('Outflow (m^3/s)')
ax3.grid(True)
ax3.legend()

plt.tight_layout()
plt.show()